In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.mva.*

GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
// Class switching example 4
// Example of class switching controlled by a reducible Markov chain
// In this variant the job remains either in class 2 or class 3 forever
val model = Network("mm1cs")

In [ ]:
// Block 1: nodes
val node = arrayOfNulls<Node>(3)
node[0] = Delay(model, "Queue 0")
node[1] = Delay(model, "Queue 1")
node[2] = Delay(model, "Queue 2")

// Block 2: classes
val jobclass = arrayOfNulls<ClosedClass>(3)
jobclass[0] = ClosedClass(model, "Class1", 1, node[0]!!)
jobclass[1] = ClosedClass(model, "Class2", 0, node[0]!!)
jobclass[2] = ClosedClass(model, "Class3", 0, node[0]!!)

In [ ]:
// Service configurations
node[0]!!.setService(jobclass[0]!!, Exp.fitMean(1.0))  // (Queue 0,Class1)
node[0]!!.setService(jobclass[1]!!, Exp.fitMean(1.0))  // (Queue 0,Class2)
node[0]!!.setService(jobclass[2]!!, Exp.fitMean(1.0))  // (Queue 0,Class3)
node[1]!!.setService(jobclass[1]!!, Exp.fitMean(1.0))  // (Queue 1,Class2)
node[2]!!.setService(jobclass[2]!!, Exp.fitMean(1.0))  // (Queue 2,Class3)

In [ ]:
// Routing matrix with class switching - following MATLAB specification exactly
val P = model.initRoutingMatrix()  // initialize routing matrix
P.set(jobclass[0], jobclass[0], node[0], node[0], 0.2)  // P{1,1}(1,1) = 0.2
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)  // P{2,2}(1,2) = 1.0
P.set(jobclass[2], jobclass[2], node[0], node[2], 1.0)  // P{3,3}(1,3) = 1.0
P.set(jobclass[0], jobclass[1], node[0], node[1], 0.3)  // P{1,2}(1,2) = 0.3
P.set(jobclass[0], jobclass[2], node[0], node[2], 0.5)  // P{1,3}(1,3) = 0.5
P.set(jobclass[1], jobclass[1], node[1], node[0], 1.0)  // P{2,2}(2,1) = 1
P.set(jobclass[2], jobclass[2], node[2], node[0], 1.0)  // P{3,3}(3,1) = 1
model.link(P)

In [ ]:
// Solve
val solver = MVA(model)
val avgTable = solver.avgChainTable
avgTable.print()